In [1]:
#global alignment (Needleman - Wunsch algorithm) 
def seqMatch(h1_list, i, h2_list, j, window):
    for w in range(0, window):
        api = h1_list[i+w]
        a = h2_list[j+w]
        if api != a:
            return 0
    return 1

def pairwise_NW(h1_list ,h2_list, dp_matched, dp_mismatched, dp_gap, isret, offset, window):
# def pairwise_NW(h1,h2,dp_matched,dp_mismatched,dp_gap,isret):
#     h1_list = featureProfile[h1]
#     h2_list = featureProfile[h2]

    #isret == 1 add return to list else not add
    if not isret:
        h1_list = [(t,a[:a.rindex('#')+1]) for t,a in h1_list]
        h2_list = [(t,a[:a.rindex('#')+1]) for t,a in h2_list]

    if len(h1_list) >= len(h2_list):
        long_list = h1_list
        short_list = h2_list
#         long_name = h1
#         short_name = h2
    else:
        long_list = h2_list
        short_list = h1_list
#         long_name = h2
#         short_name = h1

    x_lim = y_lim = len(long_list)

    # calc: Dot Matrix
    x = []
    y = []
    i = 0 #index
    matchedCount = 0
    totalCount = 0
    for api in short_list:
        j = 0 #index
        if offset + i + window > len(short_list):
            break
        for a in long_list:
            if offset + j + window > len(long_list):
                break
            #if a == api:
            if seqMatch(short_list, offset+i, long_list, offset+j, window):
                x.append(offset+i)
                y.append(offset+j)
                matchedCount += 1 # calc: density
            totalCount += 1 # calc: density
            j += 1
            if j == y_lim: break
        i += 1
        if i == x_lim: break

    # calc: dynamic programming, Needleman-Wunsch algorithm, use long_list
    #,short_list to find best local alignment # MIKE:20141023
    m = len(short_list)
    n = len(long_list)

    #MIKE: score matrix, all zero, dynamic programming needs one more row and col
    score = [[0 for j in range(n+1)] for i in range(m+1)]
    #NEWMAN: NW score matrix, initialization
    score[0] = [0+dp_gap*j for j in range(n+1)]
    for cnt in range(m+1):
        score[cnt][0] = 0+dp_gap*cnt

    #MIKE: map matched dots to a new matrix first, it is more efficient when calculating
    matched = [[0 for j in range(n+1)] for i in range(m+1)]
    for (i,j) in zip(x,y):
        matched[i+1][j+1] = 1

    # dynamic programming
    highest_score = 0
    highest_xy = (0,0)
    walk = [[0 for j in range(n+1)] for i in range(m+1)]
    for i in range(1, m+1): #start from 1, the first additional col/row are all zero, no need to process
        for j in range(1, n+1):

            # case 1: matched or mismatched
            sc_ab = 0
            if matched[i][j] == 1: # matched dot from zip(x,y)
                sc_ab = score[i-1][j-1] + dp_matched
            else:
                sc_ab = score[i-1][j-1] + dp_mismatched

            # case 2: gap y
            max_y = -100000             #===== 0->-10000
            for wy in range(1,j): # 1~j-1
                test = score[i][wy] + (j-wy) * dp_gap
                max_y = test if test > max_y else max_y

            # case 3: gap x
            max_x = -100000             #===== 0->-10000
            for wx in range(1,i):
                test = score[wx][j] + (i-wx) * dp_gap
                max_x = test if test > max_x else max_x

            # record path
            if max_x > max_y:
                if sc_ab >= max_x:
                    walk[i][j] = '='
                    score[i][j] = sc_ab
                else:
                    walk[i][j] = 'x'
                    score[i][j] = max_x
            else:
                if sc_ab >= max_y:
                    walk[i][j] = '='
                    score[i][j] = sc_ab
                else:
                    walk[i][j] = 'y'
                    score[i][j] = max_y
            if score[i][j] > highest_score:
                highest_score = score[i][j]
                highest_xy = (i,j)
        if i%100 == 0:
            print i,'/',m, 'finished.'
            
    #NEWMAN: NW walk matrix, correction on 1st row and 1st column
    walk[0] = ['y' for j in range(n+1)]
    for cnt in range(m+1):
        walk[cnt][0] = 'x'
                
    #alignment gap insertion #20150802
    alignment_x=[]
    alignment_y=[]
    while 1:
        if walk[m][n] == '=':
            alignment_x.append((m,short_list[m-1]))
            alignment_y.append((n,long_list[n-1]))
            m=m-1
            n=n-1
        elif walk[m][n] == 'x':
            alignment_x.append((m,short_list[m-1]))
            alignment_y.append((-1,'='))
            m=m-1
        else:
            alignment_x.append((-1,'='))
            alignment_y.append((n,long_list[n-1]))
            n=n-1
        if m==0 and n==0:
            break
            
    #distance measure
    residue_total = 0
    identity_total = 0
    for i,j in zip(alignment_x,alignment_y):
        if i==(-1,'=') or j==(-1,'='):
            pass
        else:
            residue_total+=1
            if i==j:
                identity_total+=1
    
    return identity_total, residue_total, zip(alignment_x,alignment_y)[::-1]